In [5]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as sf
import pandas as pd
import numpy as np 
import pyspark.sql.types as st

spark = SparkSession.builder.master('local').getOrCreate()

schema = st.StructType([
    st.StructField("date", st.DateType(), True),
    st.StructField("open", st.FloatType(), True),
    st.StructField("high", st.FloatType(), True),
    st.StructField("low", st.FloatType(), True),
    st.StructField("close", st.FloatType(), True),
    st.StructField("volume", st.FloatType(), True),
    st.StructField("currency", st.StringType(), True)
])

cdf = spark.read.csv("data/coffee.csv", header=True, 
schema=schema)

In [6]:
cdf.printSchema()

root
 |-- date: date (nullable = true)
 |-- open: float (nullable = true)
 |-- high: float (nullable = true)
 |-- low: float (nullable = true)
 |-- close: float (nullable = true)
 |-- volume: float (nullable = true)
 |-- currency: string (nullable = true)



In [7]:
# lower case columns 
cdf = cdf.toDF(*[c.lower() for c in cdf.columns])

# Add a column to the DataFrame where the values are the difference between 'Open' and 'Close'
# Add a column to the DataFrame where the values are the difference between 'High' and 'Low'
# Add a column to the DataFrame where the values are 'True' if the volume for that day was 100 or above, and otherwise 'False'.

cdf = cdf \
  .withColumn("oc_diff", sf.round(cdf.open - cdf.close, 4)) \
  .withColumn("hl_diff", sf.round(cdf.high - cdf.low, 4)) \
  .withColumn("over_100", cdf.volume >= 100)

# Add another column that contains the absolute values of the numbers in oc_diff
# Compute a column called net_sales which is the average of opening, high, low, and closing cost times the volume

abs_map = sf.udf(lambda x: round(abs(x), 4), st.FloatType())

cdf = cdf \
  .withColumn("abs_oc_diff", abs_map(cdf.oc_diff)) \
  .withColumn("net_sales", sf.round((cdf.open + cdf.close + cdf.high + cdf.low)/4, 4).cast(st.FloatType()))


In [8]:
cdf.printSchema()

root
 |-- date: date (nullable = true)
 |-- open: float (nullable = true)
 |-- high: float (nullable = true)
 |-- low: float (nullable = true)
 |-- close: float (nullable = true)
 |-- volume: float (nullable = true)
 |-- currency: string (nullable = true)
 |-- oc_diff: float (nullable = true)
 |-- hl_diff: float (nullable = true)
 |-- over_100: boolean (nullable = true)
 |-- abs_oc_diff: float (nullable = true)
 |-- net_sales: float (nullable = true)



In [9]:
# Find the average of the values in the column that has the absolute values of the difference between 'Open' and 'Close'.
# Find the average 'Open' value.
# Get the highest 'High' value.

cdf.createOrReplaceTempView("coffee")
result = spark.sql("""
select ROUND(AVG(open), 4) as avg_open,
       ROUND(AVG(abs_oc_diff), 4) as avg_abs_oc_diff,
       MAX(high) as max_high 
from coffee
""")
result.show()

# Get the count of values where the 'Volume' was less than 100
cdf.filter(cdf.over_100 == False).count()

+--------+---------------+--------+
|avg_open|avg_abs_oc_diff|max_high|
+--------+---------------+--------+
|126.0497|         1.7606|  306.25|
+--------+---------------+--------+



1638

In [11]:
# Write to parquet 
cdf.write.parquet("data/coffee.parquet", mode="overwrite")

In [193]:
## Bonus 1 -- Use Matplotlib to create a visualization (i.e. a chart) using the coffee data.
import matplotlib.pyplot as plt 

schema = st.StructType([
    st.StructField("date", st.DateType(), True),
    st.StructField("open", st.FloatType(), True),
    st.StructField("high", st.FloatType(), True),
    st.StructField("low", st.FloatType(), True),
    st.StructField("close", st.FloatType(), True),
    st.StructField("volume", st.FloatType(), True),
    st.StructField("currency", st.StringType(), True),
    st.StructField("oc_diff", st.FloatType(), True),
    st.StructField("hl_diff", st.FloatType(), True),
    st.StructField("over_100", st.BooleanType(), True),
    st.StructField("abs_oc_diff", st.FloatType(), True),
    st.StructField("net_sales", st.FloatType(), True)
])

cdf2 = spark.read.schema(schema).parquet("data/coffee.parquet/", header=True)

In [194]:
cdf2.show()

23/02/10 13:33:07 ERROR Executor: Exception in task 0.0 in stage 160.0 (TID 125)
org.apache.spark.sql.execution.QueryExecutionException: Parquet column cannot be converted in file file:///home/alex/deb-projects/spark-review/data/coffee.parquet/part-00000-f4214440-4d50-4230-ac65-f3d13c9a23bb-c000.snappy.parquet. Column: [abs_oc_diff], Expected: float, Found: BINARY
	at org.apache.spark.sql.errors.QueryExecutionErrors$.unsupportedSchemaColumnConvertError(QueryExecutionErrors.scala:724)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:278)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:116)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:554)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.columnartorow_nextBatch_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIt

Py4JJavaError: An error occurred while calling o3358.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 160.0 failed 1 times, most recent failure: Lost task 0.0 in stage 160.0 (TID 125) (192.168.0.33 executor driver): org.apache.spark.sql.execution.QueryExecutionException: Parquet column cannot be converted in file file:///home/alex/deb-projects/spark-review/data/coffee.parquet/part-00000-f4214440-4d50-4230-ac65-f3d13c9a23bb-c000.snappy.parquet. Column: [abs_oc_diff], Expected: float, Found: BINARY
	at org.apache.spark.sql.errors.QueryExecutionErrors$.unsupportedSchemaColumnConvertError(QueryExecutionErrors.scala:724)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:278)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:116)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:554)
	at org.apache.spark.sql.ca